# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Newport,GB,51.5877,-2.9984,55.94,79,0,20.02,clear sky
1,1,Mataura,NZ,-46.1927,168.8643,37.47,89,100,8.08,overcast clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,51.46,43,40,11.50,scattered clouds
3,3,Phillaur,IN,31.0292,75.7842,75.13,57,0,7.05,clear sky
4,4,Chuy,UY,-33.6971,-53.4616,63.72,67,90,13.53,overcast clouds
5,5,Viedma,AR,-40.8135,-62.9967,73.51,24,19,15.30,few clouds
6,6,Punta Arenas,CL,-53.1500,-70.9167,53.71,43,75,16.11,broken clouds
7,7,Thompson,CA,55.7435,-97.8558,32.16,69,75,18.41,broken clouds
8,8,Jamestown,US,42.0970,-79.2353,67.05,45,0,11.50,clear sky
9,9,Ocos,GT,14.5094,-92.1933,83.23,72,100,4.99,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp)&\
                                      (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Phillaur,IN,31.0292,75.7842,75.13,57,0,7.05,clear sky
4,4,Chuy,UY,-33.6971,-53.4616,63.72,67,90,13.53,overcast clouds
5,5,Viedma,AR,-40.8135,-62.9967,73.51,24,19,15.30,few clouds
8,8,Jamestown,US,42.0970,-79.2353,67.05,45,0,11.50,clear sky
10,10,Mahebourg,MU,-20.4081,57.7000,71.89,73,40,8.05,scattered clouds
12,12,Cape Town,ZA,-33.9258,18.4232,60.10,62,0,5.99,clear sky
15,15,Pagudpud,PH,18.5616,120.7868,78.94,85,23,11.36,few clouds
16,16,Puerto Madryn,AR,-42.7692,-65.0385,66.27,31,2,21.68,clear sky
22,22,Quatre Cocos,MU,-20.2078,57.7625,71.71,69,57,13.31,broken clouds
23,23,Tripoli,LY,32.8752,13.1875,74.79,58,3,15.05,clear sky


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [5]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Phillaur,IN,31.0292,75.7842,75.13,57,0,7.05,clear sky
4,4,Chuy,UY,-33.6971,-53.4616,63.72,67,90,13.53,overcast clouds
5,5,Viedma,AR,-40.8135,-62.9967,73.51,24,19,15.30,few clouds
8,8,Jamestown,US,42.0970,-79.2353,67.05,45,0,11.50,clear sky
10,10,Mahebourg,MU,-20.4081,57.7000,71.89,73,40,8.05,scattered clouds
12,12,Cape Town,ZA,-33.9258,18.4232,60.10,62,0,5.99,clear sky
15,15,Pagudpud,PH,18.5616,120.7868,78.94,85,23,11.36,few clouds
16,16,Puerto Madryn,AR,-42.7692,-65.0385,66.27,31,2,21.68,clear sky
22,22,Quatre Cocos,MU,-20.2078,57.7625,71.71,69,57,13.31,broken clouds
23,23,Tripoli,LY,32.8752,13.1875,74.79,58,3,15.05,clear sky


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [6]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City","Country", "Max Temp", "Current Description","Lat","Lng"]].copy()


# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
3,Phillaur,IN,75.13,clear sky,31.0292,75.7842
4,Chuy,UY,63.72,overcast clouds,-33.6971,-53.4616
5,Viedma,AR,73.51,few clouds,-40.8135,-62.9967
8,Jamestown,US,67.05,clear sky,42.0970,-79.2353
10,Mahebourg,MU,71.89,scattered clouds,-20.4081,57.7000
12,Cape Town,ZA,60.10,clear sky,-33.9258,18.4232
15,Pagudpud,PH,78.94,few clouds,18.5616,120.7868
16,Puerto Madryn,AR,66.27,clear sky,-42.7692,-65.0385
22,Quatre Cocos,MU,71.71,broken clouds,-20.2078,57.7625
23,Tripoli,LY,74.79,clear sky,32.8752,13.1875


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [7]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Phillaur,IN,75.13,clear sky,31.0292,75.7842,
4,Chuy,UY,63.72,overcast clouds,-33.6971,-53.4616,
5,Viedma,AR,73.51,few clouds,-40.8135,-62.9967,
8,Jamestown,US,67.05,clear sky,42.0970,-79.2353,
10,Mahebourg,MU,71.89,scattered clouds,-20.4081,57.7000,
12,Cape Town,ZA,60.10,clear sky,-33.9258,18.4232,
15,Pagudpud,PH,78.94,few clouds,18.5616,120.7868,
16,Puerto Madryn,AR,66.27,clear sky,-42.7692,-65.0385,
22,Quatre Cocos,MU,71.71,broken clouds,-20.2078,57.7625,
23,Tripoli,LY,74.79,clear sky,32.8752,13.1875,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [8]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [9]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params = params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Phillaur,IN,75.13,clear sky,31.0292,75.7842,Prithvi Resorts
4,Chuy,UY,63.72,overcast clouds,-33.6971,-53.4616,Nuevo Hotel Plaza
5,Viedma,AR,73.51,few clouds,-40.8135,-62.9967,Hotel Cristal
8,Jamestown,US,67.05,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
10,Mahebourg,MU,71.89,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
12,Cape Town,ZA,60.10,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
15,Pagudpud,PH,78.94,few clouds,18.5616,120.7868,Saud Beach Resort and Hotel - 2
16,Puerto Madryn,AR,66.27,clear sky,-42.7692,-65.0385,Hotel Tolosa
22,Quatre Cocos,MU,71.71,broken clouds,-20.2078,57.7625,Sunrise Attitude Hotel
23,Tripoli,LY,74.79,clear sky,32.8752,13.1875,Al Waddan Hotel


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [11]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Phillaur,IN,75.13,clear sky,31.0292,75.7842,Prithvi Resorts
4,Chuy,UY,63.72,overcast clouds,-33.6971,-53.4616,Nuevo Hotel Plaza
5,Viedma,AR,73.51,few clouds,-40.8135,-62.9967,Hotel Cristal
8,Jamestown,US,67.05,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
10,Mahebourg,MU,71.89,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
12,Cape Town,ZA,60.10,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
15,Pagudpud,PH,78.94,few clouds,18.5616,120.7868,Saud Beach Resort and Hotel - 2
16,Puerto Madryn,AR,66.27,clear sky,-42.7692,-65.0385,Hotel Tolosa
22,Quatre Cocos,MU,71.71,broken clouds,-20.2078,57.7625,Sunrise Attitude Hotel
23,Tripoli,LY,74.79,clear sky,32.8752,13.1875,Al Waddan Hotel


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [12]:
# In case of an empty DataFrame, load the sample data provided
#clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
#clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [13]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [15]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [16]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Prithvi Resorts</dd>\n<dt>City</dt><dd>Phillaur</dd>\n<dt>Country</dt><dd>IN</dd>\n<dt>Current Weather</dt><dd>clear sky and 75.13 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Nuevo Hotel Plaza</dd>\n<dt>City</dt><dd>Chuy</dd>\n<dt>Country</dt><dd>UY</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 63.72 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Cristal</dd>\n<dt>City</dt><dd>Viedma</dd>\n<dt>Country</dt><dd>AR</dd>\n<dt>Current Weather</dt><dd>few clouds and 73.51 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hampton Inn & Suites Jamestown</dd>\n<dt>City</dt><dd>Jamestown</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>clear sky and 67.05 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Shandrani Beachcomber Resort & Spa</dd>\n<dt>City</dt><dd>Mahebourg</dd>\n<dt>Country</dt><dd>MU</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 71.89 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Radisson

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [17]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
3,31.0292,75.7842
4,-33.6971,-53.4616
5,-40.8135,-62.9967
8,42.0970,-79.2353
10,-20.4081,57.7000
12,-33.9258,18.4232
15,18.5616,120.7868
16,-42.7692,-65.0385
22,-20.2078,57.7625
23,32.8752,13.1875


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [43]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))